About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [44]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [45]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-processing

In [47]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/sample_data/train.csv', on_bad_lines='skip', engine='python')

In [48]:
news_dataset.shape

(808, 5)

In [49]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [50]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,21
author,85
text,2
label,0


In [51]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [52]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [53]:
print(news_dataset['content'])

0      Darrell Lucus House Dem Aide: We Didn’t Even S...
1      Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2      Consortiumnews.com Why the Truth Might Get You...
3      Jessica Purkiss 15 Civilians Killed In Single ...
4      Howard Portnoy Iranian woman jailed for fictio...
                             ...                        
803    Richard C. Paddock and Rick Gladstone North Ko...
804    newsguy76 SURGEONS ADMIT THAT MAMMOGRAPHY IS O...
805                Le terrorisme régional protège Israël
806    wmw_admin WAR ON THE STREETS OF PARIS: Armed m...
807    Jerome Hudson Oscar Voter: Meryl Streep Only N...
Name: content, Length: 808, dtype: object


In [54]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [55]:
print(X)
print(Y)

      id                                              title  \
0      0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1      1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2      2                  Why the Truth Might Get You Fired   
3      3  15 Civilians Killed In Single US Airstrike Hav...   
4      4  Iranian woman jailed for fictional unpublished...   
..   ...                                                ...   
803  803  North Korean Is Arrested in Killing of Kim Jon...   
804  804  SURGEONS ADMIT THAT MAMMOGRAPHY IS OUTDATED AN...   
805  805              Le terrorisme régional protège Israël   
806  806  WAR ON THE STREETS OF PARIS: Armed migrants fi...   
807  807  Oscar Voter: Meryl Streep Only Nominated for A...   

                                    author  \
0                            Darrell Lucus   
1                          Daniel J. Flynn   
2                       Consortiumnews.com   
3                          Jessica Purkiss   
4            

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [56]:
port_stem = PorterStemmer()

In [57]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [58]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [59]:
print(news_dataset['content'])

0      darrel lucu hous dem aid even see comey letter...
1      daniel j flynn flynn hillari clinton big woman...
2                 consortiumnew com truth might get fire
3      jessica purkiss civilian kill singl us airstri...
4      howard portnoy iranian woman jail fiction unpu...
                             ...                        
803    richard c paddock rick gladston north korean a...
804    newsguy surgeon admit mammographi outdat harm ...
805                 le terrorism r gional prot ge isra l
806    wmw admin war street pari arm migrant fight ru...
807    jerom hudson oscar voter meryl streep nomin an...
Name: content, Length: 808, dtype: object


In [60]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [61]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire'
 'jessica purkiss civilian kill singl us airstrik identifi'
 'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri'
 'daniel nussbaum jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight'
 'alissa j rubin beno hamon win french socialist parti presidenti nomin new york time'
 'excerpt draft script donald trump q ampa black church pastor new york time'
 'megan twohey scott shane back channel plan ukrain russia courtesi trump associ new york time'
 'aaron klein obama organ action partner soro link indivis disrupt trump agenda'
 'chri tomlinson bbc comedi sketch real housew isi caus outrag'
 'amando flavio russian research discov secret nazi militari base

In [62]:
print(Y)

[1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1
 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0
 0 0 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0
 0 1 0 1 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0
 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1
 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1
 0 1 0 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0
 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 0 1 1
 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0 0
 0 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 1 1
 1 0 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 1 1
 1 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 1 1 1
 1 1 1 0 1 0 0 1 1 1 0 1 

In [63]:
Y.shape

(808,)

In [64]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [65]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8167 stored elements and shape (808, 3325)>
  Coords	Values
  (0, 60)	0.2813390068188309
  (0, 497)	0.33369002731996045
  (0, 606)	0.25699716850324433
  (0, 743)	0.31436882328282284
  (0, 783)	0.27399342595910525
  (0, 1015)	0.2900269849672363
  (0, 1416)	0.2346005779279845
  (0, 1545)	0.2676303943919765
  (0, 1720)	0.30066021085596845
  (0, 1782)	0.31436882328282284
  (0, 2633)	0.2813390068188309
  (0, 3084)	0.30066021085596845
  (1, 304)	0.31400494266222895
  (1, 383)	0.17114136854978435
  (1, 444)	0.39151127858108137
  (1, 574)	0.2254383666198149
  (1, 739)	0.30152920913020087
  (1, 1129)	0.6601779211624844
  (1, 1382)	0.21620892232313332
  (1, 3273)	0.30741981054082923
  (2, 602)	0.3326555839797769
  (2, 640)	0.4465664174314401
  (2, 1105)	0.3749685620065722
  (2, 1215)	0.363145011766783
  (2, 1906)	0.46692762825474105
  :	:
  (805, 1689)	0.37741479772953446
  (805, 2337)	0.41887669075592165
  (805, 2978)	0.4188766907559

Splitting the dataset to training & test data

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [67]:
model = LogisticRegression()

In [68]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [69]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [70]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9736842105263158


In [71]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [75]:
print('Accuracy score of the test data : ', test_data_accuracy)
print("\nClassification Report:\n", classification_report(X_test_prediction, Y_test))

Accuracy score of the test data :  0.9197530864197531

Classification Report:
               precision    recall  f1-score   support

           0       0.84      1.00      0.91        68
           1       1.00      0.86      0.93        94

    accuracy                           0.92       162
   macro avg       0.92      0.93      0.92       162
weighted avg       0.93      0.92      0.92       162



Making a Predictive System

In [73]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [74]:
print(Y_test[3])

1
